In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
## Implement GraphSAGE
import dgl.function as fn

class SAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        self.linear = nn.Linear(in_feat*2, out_feat)
        
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            
            #update_all is a message padding API
            g.update_all(message_func = fn.copy_u('h', 'm'),
                        reduce_func = fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim = 1)
            return self.linear(h_total)

In [8]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [9]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

model = GraphSAGE(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 1.949, val acc: 0.122 (best 0.122), test acc: 0.130 (best 0.130)
In epoch 5, loss: 1.866, val acc: 0.380 (best 0.380), test acc: 0.417 (best 0.417)
In epoch 10, loss: 1.708, val acc: 0.594 (best 0.594), test acc: 0.612 (best 0.612)
In epoch 15, loss: 1.471, val acc: 0.636 (best 0.636), test acc: 0.651 (best 0.651)
In epoch 20, loss: 1.169, val acc: 0.662 (best 0.662), test acc: 0.685 (best 0.685)
In epoch 25, loss: 0.843, val acc: 0.696 (best 0.696), test acc: 0.719 (best 0.719)
In epoch 30, loss: 0.548, val acc: 0.724 (best 0.724), test acc: 0.753 (best 0.753)
In epoch 35, loss: 0.328, val acc: 0.740 (best 0.740), test acc: 0.764 (best 0.764)
In epoch 40, loss: 0.189, val acc: 0.750 (best 0.754), test acc: 0.766 (best 0.767)
In epoch 45, loss: 0.109, val acc: 0.754 (best 0.754), test acc:

In [10]:
class WeightedSAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        self.linear = nn.Linear(in_feat*2, out_feat)
        
    def forward(self, g, h, w):
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            g.update_all(message_func = fn.u_mul_e('h', 'w', 'm'),
                        reduce_func = fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)